# ROI Normalization
This pipeline contains the basic functions to loop over the previously done annotations,and  extract the respective ROIs and Reference areas. Once all ROIS and Reference Areas are extracted, normalization can be applied.

In [14]:
import numpy as np
import pandas as pd
import os

In [15]:
def combine_roi(path):
    # Initialize an empty DataFrame to store all combined data
    df_all = pd.DataFrame()

    # Iterate over each file in the specified directory
    for filename in os.listdir(path):
        # Check if the file is a CSV file
        if filename.endswith('.csv'):
            # Construct the full file path
            file_path = os.path.join(path, filename)
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            # Concatenate the current DataFrame with the cumulative DataFrame
            df_all = pd.concat([df_all, df], ignore_index=True)
    
    # Return the combined DataFrame containing data from all CSV files
    return df_all


In [16]:
def load_roi_ref(df_all, i, current_path):
    # Extract the i-th row from the DataFrame
    df_i = df_all.loc[i]

    # Retrieve the relevant values from the DataFrame row
    uid = df_i['uid']
    handheld_type = df_i['handheld_type']
    frame = df_i['frame']
    roi_v1, roi_v2, roi_h1, roi_h2 = df_i['v1'], df_i['v2'], df_i['h1'], df_i['h2']
    ref_v1, ref_v2, ref_h1, ref_h2 = df_i['v1_new'], df_i['v2_new'], df_i['h1_new'], df_i['h2_new']

    # Construct the file path to the .npy file using the provided UID and handheld_type
    path = os.path.join(current_path, "pipeline-main", "pipeline-main", "RAW_DATA", f"{uid}", "Rohdaten", 
                        "raw_0_0.tar_extracted", f"{handheld_type}_large_rf.raw.rf.npy")
    
    # Load the 3D numpy array from the specified file path
    array3d = np.load(path)
    
    # Extract the 2D slice corresponding to the specified frame
    array2d = array3d[:,:,frame]
    
    # Slice the array to get the ROI using the specified coordinates
    roi = array2d[roi_h1:roi_h2, roi_v1:roi_v2]
    
    # Slice the array to get the reference region using the specified coordinates
    reference = array2d[ref_h1:ref_h2, ref_v1:ref_v2]
    
    # Return the ROI and reference slices
    return roi, reference


In [18]:
current_path = os.getcwd()
csv_path = os.path.join(current_path, "pipeline-main", "pipeline-main", "testrun_1")

df_all = combine_roi(csv_path)
roi, ref = load_roi_ref(df_all, 0, current_path)

print(f"ROI shape: {roi.shape}")
print(f"Reference shape: {ref.shape}")

ROI shape: (36, 297)
Reference shape: (36, 297)
